# Climate Model and Climate data workshop, hands-on session

This jupyter notebook is part of the climate model and climate data workshop. It showcases how to use climate model data in R. However, it is not a R course!

Use kernel Teaching (R SLAC 2021).

Here we will use the ncdf4 (https://search.r-project.org/CRAN/refmans/ncdf4/html/00Index.html) library.

## Loading and inspecting data

In [ ]:
#-----------------------------------------------------------------------------#
# Load libraries
#-----------------------------------------------------------------------------#
library(ncdf4)

In [ ]:
# get working directory and load function eurocentric.r
WD <- getwd()

# define directory where data is
indir <- "/net/co2/c2sm-data/rlorenz/climate_model_data_workshop/"
# define output directory 
outdir <- paste(WD, "/data/", sep="")
print(outdir)

In [ ]:
#-----------------------------------------------------------------------------#
# Read data
#-----------------------------------------------------------------------------#
file <- paste(indir, "tx_0.25deg_reg_v15.0_flt_2003_CEU.nc", sep="")
ex.nc <- nc_open(file)
print(paste("The file has",ex.nc$nvars,"variables"))
print(ex.nc)

In [ ]:
# This illustrates how to read all the data from a variable if you do not know the variable name
v1 <- ex.nc$var[[1]]
data1 <- ncvar_get( ex.nc, v1 )	# by default, reads ALL the data

print(paste("Var 1 has name",v1$name,"and",v1$ndim, "dimensions.",
    "Dimensions are:"))
print(v1$varsize)

In [ ]:
# Read variable "tx" directly if variable name is known
tx <- ncvar_get(ex.nc, "tx")

# Read longitude and latitude dimension
lon <- ncvar_get(ex.nc, "longitude")
lat <- ncvar_get(ex.nc, "latitude")
nlon <- dim(lon)                 #number of longitudes, do only if needed
nlat <- dim(lat)                 #number of latitudes, 


In [ ]:
#Read time dimension
time <- ncvar_get(ex.nc, "time")
time_units <- ncatt_get(ex.nc, "time", "units") #time unit, for how defined
# extract base date, third bit of time_units "days since 1950-01-01 00:00"
base_date <- as.character(lapply(strsplit(as.character(time_units$value),
                                          split = " "), "[", 3))
time_d <- as.Date(time, format = "%j",
                  origin = as.Date(base_date)) # define time as date
time.y <- format(time_d, "%Y") #extract years only from time_d

Y.Start <- time.y[1] # first year in timeseries
                     # -> can be used as info for output file
Y.End <- time.y[length(time.y)] # last year in timeseries

In [ ]:
# print start and end years, convert to integers
print(Y.Start)
print(Y.End)
first_year <- strtoi(Y.Start, base = 0L)
last_year <- strtoi(Y.End, base = 0L)

In [ ]:
#-----------------------------------------------------------------------------#
# Perform calculations, e.g. subset a season or find yearly summer maximum
#-----------------------------------------------------------------------------#

In [ ]:
#-----------------------------------------------------------------------------#
# save as NetCDF output
#-----------------------------------------------------------------------------#
print("save data to netcdf")

#define dimensions for output file
dim1 <- ncdim_def("longitude", "degrees_east", as.double(lon))
dim2 <- ncdim_def("latitude", "degrees_north", as.double(lat))
dimT <- ncdim_def("time", time_units$value, time, unlim = FALSE)
missval <- -9999

# define the EMPTY netcdf variable "TX"
var1 <- ncvar_def("tx", "degC", list(dim1, dim2, dimT), missval,
                  longname = "maximum temperature")

# associate the netcdf variable with a netcdf file  
file.out <- paste(outdir, "TX_", Y.Start, "-", Y.End, ".nc", sep = "")
print(file.out)
# in case the output directory does not exist yet, create it
if (!file.exists(outdir)){
   dir.create(file.path(outdir), showWarnings = FALSE)
}

# create the netcdf file
nc.ex = nc_create(file.out, list(var1))

# put data into file
ncvar_put(nc.ex, var1, tx) # one could write a subset of the data using
                           # start = c(1, 1, 1),  count = c(nlon, nlat, t2))
ncatt_put(nc.ex , var1, '_FillValue', missval )

# close the file
nc_close(nc.ex)